# <div align="center" style="color: #C64242;">**Employement Data with CIC Features and Customer Demographic**</div>

DC, for the Employment data analysis Please do the Data preparatin as following:

Get the following fields from the CICdata

  customer_id, TIN, run_date, occupationdesc, occupationstatusdesc, TradeName, employment_duration_in_days (DateHiredTo or run_date - DateHiredFrom), PSICdesc, AnnualMonthlyIndicator, Currency, GrossIncome, max_sanctioned_amt_24m, max_creditcard_limit_24m, max_financed_amt_24m, tot_granted_contracts_cnt_24m, tot_granted_contracts_amt_12m, max_sanctioned_amt_12m, Max_CreditCard_limit_12m, max_financed_amt_12m, tot_granted_contracts_cnt_12m, tot_granted_contracts_amt_12m
  
  Get the self declared employment details, application_date,  and MaxSalary information from our internal Salary scaling table using the CustomerID or TIN and application_date = run_date
  
  Get the gender, age, region, province of the customer from demographic table using customer_id or TIN and application_date = run_date
  
  Get the following fields from the Credolab tables using the customer_id or mobile number and application_date = run date:
  Device Price
  Device Manufacturer (Apple/ Samsung/ Oppo etc.)
  Device Model
  Device Age (usually derived by the application_date - earliest apps installation date)
  number of job search apps installed (linkedin, jobstreet, glint etc.)
  number of professional apps installed (office, gdrive etc)
  number of ecommerce apps installed (lazada, shopee, shein, zalora, temu etc.)
  number of ride hailing apps installed (grab, moveit etc.)
  number of finance apps installed (Banking apps, FinTech apps, eWallet apps)
  number of Risky apps (gambling apps, crypto apps, payday loan apps)
  number of OTT apps (Netflix, PrimeVideo etc.)
  Sowndarya: may be you can share your earlier apps categorisation work with DC for this. To keep things simple, just give him your final app categorisation mapping.

DC, next steps on the Income data:
Tradename column from CICdata is missing, pls add it
Calculate the employment duration (if no end date then run date - start date, otherwise end date - start date, if no start date then null)
Create two additional flags from CICdata:
--> Has Credit Loan (credit_loan_flag)
--> Has Installment loan flag (inst_loan_flag)
Add the other Credolab fields:
--> App category wise count (just use Credolab app categories)
-->  Device Brand (from credolab_raw.android_credolab_datasets_struct_columns` select Hardware.device_brand)
--> Device Model (from credolab_raw.android_credolab_datasets_struct_columns` select Hardware.device_model)
--> Network Provider (from credolab_raw.android_credolab_datasets_struct_columns` select GeneralData.sim_operator_name)
--> Number of BlueTooth Device (from credolab_raw.android_credolab_datasets_struct_columns` select Hardware.bluetooth__bonded_devices_count)

In [22]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import timeit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
client = bigquery.Client(project='prj-prod-dataplatform')
import re
from fuzzywuzzy import process
# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## ***Combining the Granted and Non Granted tables data***

## <div style="color: #C64242; font-family: Monotype Corsiva;"><b> Main Query Part </b> </div>

In [9]:
%%bigquery employmentdata

select * from prj-prod-dataplatform.risk_credit_cic_data.employment_data limit 10;

Query is running:   0%|          |

Downloading:   0%|          |

In [10]:
employmentdata.head()

,digitalLoanAccountId,crifApplicationId,customerId,processEngineGuid,requestGuid,LastUpdateDate,AnnualMonthlyIndicator,Currency,DateHiredFrom,DateHiredTo,...,Occupation,OccupationStatus,PhoneNumber,PSIC,TIN,CurrencyDesc,OccupationDesc,OccupationStatusDesc,PSICDesc,TradeName
0,d3b65acb-2d39-43c2-bbea-b708dfade943,tonik-tul-474890,2443782,01972dc0-ecd8-11ee-b07b-0242ace6000f,0177e5f0-ecd8-11ee-989e-0242ace60004,2022-10-20,None,None,None,None,...,None,None,None,None,None,None,None,None,None,CAFE BRETON
1,aa17e1da-2ad6-4766-9eff-6c19c5c9be91,tonik-tul-475115,2444117,aada32b0-ed05-11ee-a41d-0242ace60015,aaac44e0-ed05-11ee-a4cc-0242ace60008,2019-11-30,Y,PHP,2016-03-21,None,...,None,3,None,None,None,Philippine peso,None,Permanent Job (Government sector),None,"DEPED DIV OFC, CADIZ"
2,951cea6d-47d8-41fa-9996-239b31c01d05,tonik-bnpl-474873,2443767,76941730-ecd5-11ee-b07b-0242ace6000f,767348c0-ecd5-11ee-989e-0242ace60004,2019-11-30,Y,PHP,1999-09-01,None,...,None,3,None,None,None,Philippine peso,None,Permanent Job (Government sector),None,CITY GOVT OF CAVITE
3,9a062ab8-2c7b-46c8-940f-e732558f6588,tonik-bnpl-475089,2444065,a66358f0-ecf7-11ee-b07b-0242ace6000f,a6452290-ecf7-11ee-989e-0242ace60004,2023-06-20,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,3471ef45-052c-41f8-b628-12d0c091af97,tonik-bnpl-475062,2444020,f7549b00-ecef-11ee-a41d-0242ace60015,f72b1a00-ecef-11ee-a4cc-0242ace60008,2024-02-29,None,None,None,None,...,None,None,7158516,None,None,None,None,None,None,GLOBAL VENTURE PROMOTIONS AND MARKETING SERVICES


In [11]:
employmentdata['TradeName'].value_counts()

,count
TradeName,
CAFE BRETON,1
"DEPED DIV OFC, CADIZ",1
CITY GOVT OF CAVITE,1
GLOBAL VENTURE PROMOTIONS AND MARKETING SERVICES,1
ROMBLON STATE UNIVERSITY,1
CLEVELAND TOWER CONDO CORP,1
CHERRY GOLD PABAZA INC,1


In [12]:
%%bigquery df1

select distinct ContractHistoryType from prj-prod-dataplatform.risk_credit_cic_data.granted_contracts

Query is running:   0%|          |

Downloading:   0%|          |

In [13]:
df1

,ContractHistoryType
0,Installments
1,CreditCards
2,Utilities
3,NonInstallments


In [14]:
%%time

# getting the customer demographic information from loan master table and employment information from cic employment data table

sq ="""with base as
(
-- Query for dfgranted
SELECT
  digitalLoanAccountId,
  crifApplicationId,
  customerId,
  processEngineGuid,
  requestGuid,
  ContractHistoryType,
  CBContractCode,
  ContractEndDate,
  ContractPhase,
  ContractPhaseDesc,
  ContractStartDate,
  ContractStatus,
  ContractStatusDesc,
  ContractType,
  ContractTypeDesc,
  Currency,
  CurrencyDesc,
  LastUpdateDate,
  OriginalCurrency,
  OriginalCurrencyDesc,
  ProviderCodeEncrypted,
  ProviderContractNo,
  ReferenceNo,
  Role,
  RoleDesc,
  BilledAmount,
  BoardResolutionFlag,
  BoardResolutionFlagDesc,
  CancellationDate,
  CardReferenceCode,
  ChargedAmount,
  CreditLimit,
  CreditPurpose,
  CreditPurposeDesc,
  FinancedAmount,
  FirstPaymentDate,
  FlagCardUsed,
  HolderLiability,
  HolderLiabilityDesc,
  InstallmentType,
  InstallmentTypeDesc,
  InstallmentsNumber,
  LastChargeDate,
  LastPaymentAmount,
  LastPaymentDate,
  MinPaymentIndicator,
  MinPaymentIndicatorDesc,
  MinPaymentPercentage,
  MonthlyPaymentAmount,
  NextPayment,
  NextPaymentDate,
  OutstandingBalance,
  OutstandingBalanceUnbilled,
  OutstandingPaymentsNumber,
  OverallCreditLimit,
  OverdueDays,
  OverdueDaysDesc,
  OverduePaymentsAmount,
  OverduePaymentsNumber,
  PaymentMethod,
  PaymentMethodDesc,
  PaymentPeriodicity,
  PaymentPeriodicityDesc,
  PremiumCard,
  PremiumCardDesc,
  ReorganizedCreditCode,
  ReorganizedCreditCodeDesc,
  ServicesLinesNo,
  TimesCardUsed,
  TransactionType,
  TransactionTypeDesc,
  Utilization,
  LinkedSubject_CBSubjectCode,
  LinkedSubject_Name,
  LinkedSubject_Role,
  LinkedSubject_RoleDesc,
  Note_TypeDesc,
  Note_Text,
  Note_Type,
  run_date,
  NULL AS ContractRequestDate,
  'granted' AS source
FROM
  prj-prod-dataplatform.risk_credit_cic_data.granted_contracts

UNION ALL

-- Query for dfnongranted
SELECT
  digitalLoanAccountId,
  crifApplicationId,
  customerId,
  processEngineGuid,
  requestGuid,
  NULL AS ContractHistoryType,
  CBContractCode,
  NULL AS ContractEndDate,
  ContractPhase,
  ContractPhaseDesc,
  NULL AS ContractStartDate,
  NULL AS ContractStatus,
  NULL AS ContractStatusDesc,
  ContractType,
  ContractTypeDesc,
  NULL AS Currency,
  NULL AS CurrencyDesc,
  LastUpdateDate,
  NULL AS OriginalCurrency,
  NULL AS OriginalCurrencyDesc,
  ProviderCodeEncrypted,
  ProviderContractNo,
  ReferenceNo,
  Role,
  RoleDesc,
  NULL AS BilledAmount,
  NULL AS BoardResolutionFlag,
  NULL AS BoardResolutionFlagDesc,
  NULL AS CancellationDate,
  NULL AS CardReferenceCode,
  NULL AS ChargedAmount,
  CreditLimit,
  NULL AS CreditPurpose,
  NULL AS CreditPurposeDesc,
  FinancedAmount,
  NULL AS FirstPaymentDate,
  NULL AS FlagCardUsed,
  NULL AS HolderLiability,
  NULL AS HolderLiabilityDesc,
  NULL AS InstallmentType,
  NULL AS InstallmentTypeDesc,
  InstallmentsNumber,
  NULL AS LastChargeDate,
  NULL AS LastPaymentAmount,
  NULL AS LastPaymentDate,
  NULL AS MinPaymentIndicator,
  NULL AS MinPaymentIndicatorDesc,
  NULL AS MinPaymentPercentage,
  MonthlyPaymentAmount,
  NULL AS NextPayment,
  NULL AS NextPaymentDate,
  NULL AS OutstandingBalance,
  NULL AS OutstandingBalanceUnbilled,
  NULL AS OutstandingPaymentsNumber,
  NULL AS OverallCreditLimit,
  NULL AS OverdueDays,
  NULL AS OverdueDaysDesc,
  NULL AS OverduePaymentsAmount,
  NULL AS OverduePaymentsNumber,
  NULL AS PaymentMethod,
  NULL AS PaymentMethodDesc,
  PaymentPeriodicity,
  PaymentPeriodicityDesc,
  NULL AS PremiumCard,
  NULL AS PremiumCardDesc,
  NULL AS ReorganizedCreditCode,
  NULL AS ReorganizedCreditCodeDesc,
  NULL AS ServicesLinesNo,
  NULL AS TimesCardUsed,
  NULL AS TransactionType,
  NULL AS TransactionTypeDesc,
  NULL AS Utilization,
  LinkedSubject_CBSubjectCode,
  LinkedSubject_Name,
  LinkedSubject_Role,
  LinkedSubject_RoleDesc,
  Note_TypeDesc,
  Note_Text,
  Note_Type,
  run_date,
  ContractRequestDate,
  'nongranted' AS source
FROM
  prj-prod-dataplatform.risk_credit_cic_data.notgranted_contracts
),
base2 as
(select distinct * from base),
loanmaster as --- data from loan master for customer demographic details
(select distinct customerId, digitalLoanAccountId, loanAccountNumber, startApplyDateTime, natureofwork,industryDescription, incomeMultiple, employmentstatus,credolabScore,city,
       barangay, province, postalCode, age, monthlyIncome,  tinNumber, gender, dateOfBirth, email, mobileNo,firstName, middleName, lastName, custLoanJourneyZipCode,
       custLoanJourneyProvince, custLoanJourneyCity,  tsa_onboarding_time,new_loan_type
 from prj-prod-dataplatform.risk_credit_mis.loan_master_table),
employmentdata as ---- data from employment table for employment related information
(SELECT distinct
  digitalLoanAccountId,
  crifApplicationId,
  customerId,
  AnnualMonthlyIndicator,
  Currency,
  DateHiredFrom,
  DateHiredTo,
  case when DateHiredFrom is null then null
       when DateHiredFrom is not null and  DateHiredTo is not null then date_diff(date(DateHiredTo), date(DateHiredFrom), month)
       when DateHiredFrom is not null and DateHiredTo is null then date_diff(date((select min(run_date) from base2 where base2.digitalLoanAccountId = digitalLoanAccountId)), date(DateHiredFrom), month)
       else null end Employment_duration,
  GrossIncome,
  CAST(
    CASE
      WHEN COALESCE(AnnualMonthlyIndicator, 'NA') LIKE 'M' THEN CAST(COALESCE(GrossIncome, '0') AS NUMERIC)
      WHEN COALESCE(AnnualMonthlyIndicator, 'NA') LIKE 'Y' THEN ROUND(CAST(COALESCE(GrossIncome, '0') AS NUMERIC)/12, 0)
      ELSE 0
    END AS INT64
  ) AS MonthlyIncomecalculated,
  CAST(
    CASE
      WHEN COALESCE(AnnualMonthlyIndicator, 'NA') LIKE 'M' THEN ROUND(CAST(COALESCE(GrossIncome, '0') AS NUMERIC)*12, 0)
      WHEN COALESCE(AnnualMonthlyIndicator, 'NA') LIKE 'Y' THEN CAST(COALESCE(GrossIncome, '0') AS NUMERIC)
      ELSE 0
    END AS INT64
  ) AS AnnualIncomecalculated,
  Occupation,
  OccupationDesc,
  OccupationStatus,
  OccupationStatusDesc,
  TIN,
  PhoneNumber,
  PSIC,
  REGEXP_REPLACE(PSICDesc, r'^\d+\s*-\s*', '') AS PSICDesc ,
  TradeName,
  Case when digitalLoanAccountId in (select digitalLoanAccountId from base2 where base2.digitalLoanAccountid = digitalLoanAccountId and ContractHistoryType like 'Installments') then 1 else 0 end installmentLoanFlag,
  Case when digitalLoanAccountId in (select digitalLoanAccountId from base2 where base2.digitalLoanAccountid = digitalLoanAccountId and ContractHistoryType like '%CreditCards%' ) then 1 else 0 end creditLoanFlag,
  Case when digitalLoanAccountId in (select digitalLoanAccountId from base2 where base2.digitalLoanAccountId = digitalLoanAccountId and ContractHistoryType like 'NonInstallments' ) then 1 else 0 end noninstallmentLoanFlag,
  row_number() over (partition by digitalLoanAccountId order by digitalLoanAccountId ) as rnk
FROM prj-prod-dataplatform.risk_credit_cic_data.employment_data)
select  ---joining both the above tables
lm.customerId, lm.digitalLoanAccountId, loanAccountNumber,
       startApplyDateTime, natureofwork, industryDescription,
       incomeMultiple, employmentstatus, credolabScore, city,
       barangay, province, postalCode, age, monthlyIncome,
       tinNumber, gender, dateOfBirth, email, mobileNo, firstName,
       middleName, lastName, custLoanJourneyZipCode,
       custLoanJourneyProvince, custLoanJourneyCity, tsa_onboarding_time,
       new_loan_type, ed.digitalLoanAccountId edprimarykey, crifApplicationId,
       AnnualMonthlyIndicator, DateHiredFrom,
       DateHiredTo,Employment_duration, GrossIncome, MonthlyIncomecalculated, AnnualIncomecalculated, Occupation, OccupationDesc,
       OccupationStatus, OccupationStatusDesc, TIN, PhoneNumber,
       PSIC, PSICDesc, TradeName , installmentLoanFlag, creditLoanFlag, noninstallmentLoanFlag
from loanmaster lm
inner join (select * from employmentdata where rnk = 1) ed on ed.digitalLoanAccountId = lm.digitalLoanAccountId"""

dfdemograph_employment = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 3224269c-1e2b-45b9-a428-3c681fb8d981 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
CPU times: user 1.76 s, sys: 840 ms, total: 2.6 s
Wall time: 19.1 s


In [15]:
dfdemograph_employment.columns

Index(['customerId', 'digitalLoanAccountId', 'loanAccountNumber',
       'startApplyDateTime', 'natureofwork', 'industryDescription',
       'incomeMultiple', 'employmentstatus', 'credolabScore', 'city',
       'barangay', 'province', 'postalCode', 'age', 'monthlyIncome',
       'tinNumber', 'gender', 'dateOfBirth', 'email', 'mobileNo', 'firstName',
       'middleName', 'lastName', 'custLoanJourneyZipCode',
       'custLoanJourneyProvince', 'custLoanJourneyCity', 'tsa_onboarding_time',
       'new_loan_type', 'edprimarykey', 'crifApplicationId',
       'AnnualMonthlyIndicator', 'DateHiredFrom', 'DateHiredTo',
       'Employment_duration', 'GrossIncome', 'MonthlyIncomecalculated',
       'AnnualIncomecalculated', 'Occupation', 'OccupationDesc',
       'OccupationStatus', 'OccupationStatusDesc', 'TIN', 'PhoneNumber',
       'PSIC', 'PSICDesc', 'TradeName', 'installmentLoanFlag',
       'creditLoanFlag', 'noninstallmentLoanFlag'],
      dtype='object')

In [16]:
pd.set_option("Display.max_columns", None)
dfdemograph_employment.head()

,customerId,digitalLoanAccountId,loanAccountNumber,startApplyDateTime,natureofwork,industryDescription,incomeMultiple,employmentstatus,credolabScore,city,barangay,province,postalCode,age,monthlyIncome,tinNumber,gender,dateOfBirth,email,mobileNo,firstName,middleName,lastName,custLoanJourneyZipCode,custLoanJourneyProvince,custLoanJourneyCity,tsa_onboarding_time,new_loan_type,edprimarykey,crifApplicationId,AnnualMonthlyIndicator,DateHiredFrom,DateHiredTo,Employment_duration,GrossIncome,MonthlyIncomecalculated,AnnualIncomecalculated,Occupation,OccupationDesc,OccupationStatus,OccupationStatusDesc,TIN,PhoneNumber,PSIC,PSICDesc,TradeName,installmentLoanFlag,creditLoanFlag,noninstallmentLoanFlag
0,2659725,6111ab1c-5963-4bba-93b6-143972ca7ade,None,2024-07-14 16:09:18,Sales Marketing Professional,Manufacturing,NaN,SALARIED,None,BUTUAN CITY,TANDANG SORA POB BGY 12,Agusan del Norte,8600,31,20000,456017783,F,07 Aug 1993,alynlubay@gmail.com,639100270271,ANALYN,VAGUTCHAY,LUBAY,8600,AGUSAN DEL NORTE,BUTUAN CITY,2024-07-14 16:04:31,Quick,6111ab1c-5963-4bba-93b6-143972ca7ade,taran-2659725-5559859,None,None,None,<NA>,None,0,0,None,None,None,None,None,None,None,None,None,1,0,1
1,2629058,1808ead9-8816-448f-a839-2e397889f825,None,2024-07-03 01:28:39,Other Professional services,Services,NaN,SALARIED,432.0,DAVAO CITY,BARANGAY 31 D POB,Davao del Sur,8000,33,20000,360374524,M,02 Jul 1991,jamersali7291@gmail.com,639923850563,JAMER,HASSAN,SALI,8000,DAVAO DEL SUR,DAVAO CITY,2024-07-03 01:21:56,Quick,1808ead9-8816-448f-a839-2e397889f825,taran-2629058-6091955,None,None,None,<NA>,None,0,0,None,None,None,None,None,None,None,None,None,1,0,0
2,2573508,5350142c-4e86-4245-811d-b8987494dd31,None,2024-06-12 20:48:10,Other Professional services,Trading,NaN,SALARIED,421,DAGOHOY,SANTA CRUZ,Bohol,6322,49,15000,199065230,M,15 Dec 1974,lythztumale2017@gmail.com,639073267818,ANGELITO,LAGUMBAY,TUMALE,6322,BOHOL,DAGOHOY,2024-06-12 20:41:40,Quick,5350142c-4e86-4245-811d-b8987494dd31,tonik-tul-554819,None,None,None,<NA>,None,0,0,None,None,None,None,None,None,None,None,None,1,0,0
3,2530793,f715e137-9fb7-4c71-9393-3930094c8c72,None,2024-05-24 12:21:35,Other Professional services,Manufacturing,NaN,SALARIED,443,BINAN,SAN JOSE,Laguna,4024,34,20000,734430397,F,07 Aug 1990,jaemelyn.guico07@gmail.com,639638915756,JAEMELYN,GUICO,DUNGO,4024,LAGUNA,BINAN,2024-05-24 11:15:59,Quick,f715e137-9fb7-4c71-9393-3930094c8c72,tonik-tul-530329,None,None,None,<NA>,None,0,0,None,None,None,None,None,None,None,None,None,1,0,0
4,2531365,c062d74a-1487-4d87-833d-03f656ebb3fe,None,2024-05-24 16:25:38,Other Professional services,Services,NaN,SALARIED,475,CAINTA,SAN JUAN,Rizal,1900,29,20000,707092465,M,21 Aug 1994,lojietanglao399@gmail.com,639369703783,LOJIE,LADIA,TANGLAO,1900,RIZAL,CAINTA,2024-05-24 16:21:40,Quick,c062d74a-1487-4d87-833d-03f656ebb3fe,tonik-tul-530649,None,None,None,<NA>,None,0,0,None,None,None,None,None,None,None,None,None,1,0,0


In [17]:
dfdemograph_employment[['installmentLoanFlag', 'creditLoanFlag','noninstallmentLoanFlag']].value_counts(dropna= False)

installmentLoanFlag  creditLoanFlag  noninstallmentLoanFlag
0                    0               0                         154296
1                    0               0                          50418
                     1               0                          20641
                     0               1                          17132
                     1               1                          14335
0                    1               0                          12999
                     0               1                           3928
                     1               1                           1716
Name: count, dtype: int64

In [18]:
import pandasql as psql

In [19]:
sq = """select digitalLoanAccountId, count(digitalLoanAccountId) cnt from dfdemograph_employment group by 1 having count(digitalLoanAccountId) > 1;"""
result = psql.sqldf(sq, locals())

In [20]:
len(result)

0